# Numpy implementation of main losses and metrics

**WIP**

Implment in numpy the most common losses and metrics. Compare with tensorflow implemnattion and write a brief intro.

Try to dicuss aplication. pros and crons and properties

**GOAL:** 
 
 what
 1. equations and properties
 1. pros and crons
 1. aplications
 
 losse and metrics
 1. binary cross entropy
 1. cross entropy  
     * relation with the binary loss
 1. categorical and sparse categorical cross entropy
     * relation with cross entropy loss
 1. mae, mse and mape
     * the impact of large vs small errors (mse penalises more large error)
     * show best consant estimator (mean for mse and median for mae)

refs:


* NOTE: list of todos
* DONE: ~~understand the sparse categorical loss equation. ~~
     1. ~~Implement it using numpy.Softmax, categorical and sparse categorical cross entropy~~
     1. ~~Run some examples~~ ok
     1. ~~show the lead to same reulst and the diff is the label format~~
     1. ~~prove categorcal cross entropy = softmax + cross entropy loss~~
     1. see: https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy
             https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy
             https://gombru.github.io/2018/05/23/cross_entropy_loss/  <== looks good 
             https://stackoverflow.com/questions/59787897/how-does-tensorflow-sparsecategoricalcrossentropy-work


In [21]:
!hostname 
!ls

ds-vm-leandro2
2018-06-08-lhof-running_gmm_em_code_from_mcdickenson.ipynb
2018-06-18-lhof-simulated_data.ipynb
2021-10-31-lhof-study_learning_rate_finder.ipynb
2023-02-10-lhof-study_GANS.ipynb
intro_KNN.ipynb
intro_batch_normalization.ipynb
intro_gradient_algos.ipynb
intro_kmeans.ipynb
intro_pca.ipynb
intro_plda.ipynb
intro_reinforcement_learning.ipynb
intro_to_prediction_interval.ipynb
machine_learning_metrics.ipynb


# Intro

# Load requirements

In [1]:
import IPython
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Image

InteractiveShell.ast_node_interactivity = "all"
print("agor vai")

# import nbformat
# from nbformat import v4

agor vai


In [2]:
import tensorflow as tf
from keras.utils.vis_utils import plot_model

import numpy as np
import matplotlib.pyplot as plt

2023-03-12 07:23:43.487609: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 07:23:43.617201: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-12 07:23:43.622967: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-12 07:23:43.622984: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

```sh
pip install nbformat
```

# Understand the loss first

https://towardsdatascience.com/cross-entropy-loss-function-f38c4ec8643e#:~:text=Categorical%20cross%2Dentropy%20is%20used,%5D%20for%203%2Dclass%20problem.

The **sparse_categorical_crossentropy** loss function is similar to the categorical_crossentropy loss function, but it is designed for use with integer-encoded class labels. Here's how the sparse_categorical_crossentropy loss function works:

1. The true class labels are represented as integers, and the predicted class probabilities are output as a softmax activation.

1. The loss function calculates the cross-entropy between the true class labels and the predicted class probabilities.

1. The calculated loss value is minimized during the model training process using an optimizer.

$
L = -\frac{1}{C}\sum_{i=1}^{C}{y_{i} \log(\hat{(y_{i}})}
$


where $C$ is the number of classes, $y_{i}$ is the true class label for class $i$, and $\hat{y_{i}} = f(s_i)$ is the predicted probability for class $i$. The loss is calculated over all classes by taking the sum of the cross-entropy loss for each class.

The difference between **categorica crossentropy** and **sparse categorical corss entropy** are":

* categorica crossentropy: true labels are one-hot encoded. Ex. 3-class classification problem [1,0,0], [0,1,0] and [0,0,1].

* sparse categorical corss entropy: truth labels are integer encoded. Ex. 1,2,3

In [67]:
def cross_entropy(y_true, y_pred):

    loss = - np.sum(y_true * np.log(y_pred), axis=-1)
    
    return np.mean(loss)

def softmax(z):
    
    exp_z = np.exp(z)
    
    return exp_z / np.sum(exp_z, axis=-1, keepdims=True)

In [66]:
num_classes = 3
batch_size = 4
y_true = np.random.randint(num_classes, size=(batch_size,)).reshape(-1, 1)
y_true = np.eye(num_classes)[y_true.reshape(-1)].reshape(batch_size, num_classes)

y_true

y_pred = np.random.rand(batch_size, num_classes)
y_pred


# ground true
cce_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(y_true, y_pred)
print('Ground True Categorical crossentropy loss:', cce_loss.numpy())

loss = cross_entropy(y_true, softmax(y_pred))
print(f"Categorical Cross entropy loss: {loss}")

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])

array([[0.99431072, 0.64088268, 0.95640322],
       [0.47087578, 0.36102252, 0.37446763],
       [0.58858135, 0.81022701, 0.40814268],
       [0.51124761, 0.44928348, 0.34913449]])

Ground True Categorical crossentropy loss: 1.0190051691713176
Categorical Cross entropy loss: 1.0190051691713176


In [35]:
# NOTE: testing categorical cross entropy
cce = tf.keras.losses.CategoricalCrossentropy()

y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.00, 1.0, 0], [0.0, 0.0, 1.]]
cce(y_true, y_pred).numpy() # Expected zero


y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.01, .1, 0.1], [0.1, 0.1, 0.1]]
cce(y_true, y_pred).numpy()

1.1920929e-07

0.92027485

In [38]:
# NOTE: Showing categorical and sparce categorical croos enteopy are the same
# Using 'auto'/'sum_over_batch_size' reduction type.
scce = tf.keras.losses.SparseCategoricalCrossentropy()
cce = tf.keras.losses.CategoricalCrossentropy()

y_pred = [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]]
y_true_scce =  [0,1,2]
y_true_cce =  [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
               
scce(y_true_scce, y_pred).numpy() # expected zero
cce(y_true_cce, y_pred).numpy() # expected zero

y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
y_true_scce = [1, 2]
y_true_cce =  [[0, 1, 0], [0, 0, 1]]
scce(y_true_scce, y_pred).numpy() # expected > 0
cce(y_true_cce, y_pred).numpy() # expected > 0

2.3841854e-07

1.192093e-07

1.1769392

1.1769392

In [50]:
# NOTE: Showing that Categorical cross entropy loss = softmax + cross entropy loss
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
y_true_cce =  [[0, 1, 0], [0, 0, 1]]

# Define some example data
y_true = tf.constant([[0, 1, 0], [1, 0, 0]])
y_pred = tf.constant([[0.05, 0.95, 0], [0.9, 0.05, 0.05]])

# Compute the categorical crossentropy loss directly 
# from_logits=True tells keras to normalize the inputs with softmax 
cce_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(y_true, y_pred)
print('Categorical crossentropy loss:', cce_loss.numpy())

# Compute the softmax and then the crossentropy loss
# from_logits=Fase tells keras to DO NOT normalize the inputs 
softmax_y_pred = tf.nn.softmax(y_pred)
ce_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)(y_true, softmax_y_pred)
print('Crossentropy loss using softmax:', ce_loss.numpy())

Categorical crossentropy loss: 0.6009282
Crossentropy loss using softmax: 0.6009282


In [125]:
y_true = [1]
y_pred = [[0, 0.1, 0]]
scce(y_true, y_pred).numpy() # expected zero

1.9073468e-06